# Grupo de Estudos em Aprendizagem de Máquina e Computação Quântica (AMCQ)

**Tutorial**: Multilayer Perceptron (MLP)

**Autor**: [Karl Sousa](github.com/karlvandesman)

# Introdução

Pra entender o funcionamento de uma rede neural, vamos ver rapidamente como funciona um neurônio isoladamente. O modelo de neurônio utilizado está mostrado na figura seguinte. De forma geral, temos um conjunto de entradas sendo multiplicadas por pesos, que então são somados no neurônio, e por fim aplicada uma função de ativação para gerar a saída daquele neurônio.
<img src="./neuron_model.png" width="300"/>

No entanto, as redes com uma só camada resolvem apenas problemas linearmente separáveis, cuja solução pode ser obtida pela separação em duas regiões por meio de uma reta ou hiper-plano, no caso $n$-dimensional. Para expandir essa capacidade de resolução de problemas, podemos adicionar uma ou mais camadas intermediárias (ou escondidas). E aqui chegamos na definição de MLP (_multilayer perceptron_), que são RNAs que apresentam pelo menos uma camada intermediária ou escondida [RHW86]. Logo, sua representação simples, com apenas uma camada intermediária, é apresentada a seguir.

<img src="./nn_model.png" width="400"/>

**Equações do algoritmo**

Para um exemplo $x^{(i)}$:
$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}\tag{1}$$ 
$$a^{[1] (i)} = \tanh(z^{[1] (i)})\tag{2}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}\tag{3}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$
$$y^{(i)}_{predicao} = \begin{cases} 1 & \mbox{, se } a^{[2](i)} > 0.5 \\ 0 & \mbox{, caso contrário } \end{cases}\tag{5}$$

Função de perda $\mathcal{L}$, diferença entre o valor esperado e o calculado:
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{6}$$

O custo é então calculado pela média de perda nos exemplos de treinamento:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{7}$$

**Passos importantes**:
A metodologia geral para construir a rede neural consiste em:
1. Definir a estrutura da rede neural (# de unidades de entrada, # de unidades escondidas, etc).
1. Inicializar os parâmetros do modelo
1. Loop:
    - Implementar a propagação direta
    - Calcular o custo
    - Implementar a propagação reversa (backprop) para obter os gradientes
    - Atualizar os parâmetros (gradiente descendente, ou outra função de otimização)

Normalmente se constrói esses 3 passos separadamente e os integra em uma função, como `nn_model()`. Uma vez criado esse modelo e rodado o algoritmo para aprender os parâmetros no treinamento, pode-se fazer previsões em novos dados.

## Funções auxiliares

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def layer_sizes(X, Y):
    """
    Argumentos:
    X -- dataset de entrada de dimensões (tamanho da entrada, número de exemplos)
    Y -- rótulos de dimensões (tamanho da saída, número de exemplos)
    
    Retorna:
    n_x -- tamanho da camada de entrada
    n_h -- tamanho da camada intermediária
    n_y -- tamanho da camada de saída
    """
    n_x = X.shape[0]
    n_h = 4
    n_y = Y.shape[0]

    return (n_x, n_h, n_y)

In [3]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argumentos:
    n_x -- tamanho da camada de entrada
    n_h -- tamanho da camada intermediária
    n_y -- tamanho da camada de saída
    
    Returna:
    params -- dicionário do python contendo os parâmetros:
                    W1 -- matriz de pesos da primeira camada, dimensões (n_h, n_x)
                    b1 -- vetor de bias de dimensões (n_h, 1)
                    W2 -- matriz de pesos da segunda camada, dimensões (n_y, n_h)
                    b2 -- vetor de bias de dimensões (n_h, 1)
    """
    
    np.random.seed(2) # semente de geração de números aleatórios
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    # Checagem das dimensões
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

## Propagação direta (Fase _Forward_)

As equações para progação já foram mostradas acima, mas para recordar:

$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}$

$a^{[1] (i)} = \tanh(z^{[1] (i)}) $

$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}$

$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})$

In [4]:
def forward_propagation(X, parameters):
    """"
    Argumentos:
    X -- dados de entrada de dimensões (n_x, m)
    parameters -- dicionário python contendo os parâmetros (saída da função de inicialização)
    
    Retorna:
    A2 -- a saída da função de ativação da segunda camada
    cache -- dicionário contendo "Z1", "A1", "Z2" and "A2"
    """
    # Recuperar cada parâmetro do dicionário parâmetros
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Implementa a propagação direta para calcular A2 (probabilidades)
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = np.tanh(Z2)
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

Agora que temos o vetor de saída previsto, devemos calcular o erro e o custo (média dos erros dos exemplos):

$$J = - \frac{1}{m} \sum\limits_{i = 1}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)}$$

In [5]:
def compute_cost(A2, Y):
    """
    Calcula a função de custo (cross-entropy)
    
    Argumentos:
    A2 -- saída da função de ativação da segunda camada, de dimensões (1, número de exemplos)
    Y -- saída rotulada de dimensões (1, número de exemplos)
    parameters -- dicionário python contendo os parâmetros W1, b1, W2 e b2
    
    Retorna:
    cost -- custo da função de entropia cruzada (cross-entropy)
    
    """
    
    m = Y.shape[1] # número de exemplos

    # Compute the cross-entropy cost
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1-A2), 1-Y)
    cost = -1/m * np.sum(logprobs)
    
    cost = float(np.squeeze(cost))  # assegura as dimensões de custo 
                                    # E.g., converte [[17]] em 17 
    assert(isinstance(cost, float))
    
    return cost

## Propagação reversa (Fase _backward_)
De maneira geral, o processo de aprendizado consistem em obter o valor de incremento dos pesos da rede, de forma que o novo valor após a atualização chegue mais perto da solução desejada. Essa fase de propagação na direção reversa (saída para a entrada), trata justamente disso, partindo inicialmente da diferença entre a saída calculada e a rotulada (aprendizado supervisionado), ou seja, partindo do erro, e fazendo estimativas dos erros nas camadas anteriores da rede.

Para essa nova etapa, consideramos que:
- Já temos inicializados os valores dos parâmetros.
- Também temos calculado a função de custo e o seu gradiente.
- Agora, queremos atualizar os parâmetros por meio do gradiente descendente.

<img src="./grad_summary.png" width="500"/>

In [6]:
def backward_propagation(parameters, cache, X, Y):
    """  
    Argumentos:
    parameters -- dicionário python contendo os parâmetros (saída da função de inicialização)
    cache -- dicionário contendo "Z1", "A1", "Z2" and "A2"
    X -- dados de entrada de dimensões (2, número de exemplos)
    Y -- vetor de saída com os rótulos corretos (1, número de exemplos)
    
    Retorna:
    grads -- dicionário python contendo os gradientes em relação aos diferentes parâmetros
    """
    m = X.shape[1]
    
    # Recuperar W1 e W2 do dicionário "parâmetros"
    W1 = parameters["W1"]
    W2 = parameters["W2"]

    # Recuperar A1 e A2 do dicionário "cache"

    A1 = cache["A1"]
    A2 = cache["A2"]
    
    # Propagação reversa: calcular dW1, db1, dW2, db2 
    dZ2 = A2 - Y
    dW2 = 1/m * np.dot(dZ2, A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = 1/m * np.dot(dZ1, X.T)    
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)    
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

Após o cálculo do gradiente, chegamos enfim na etapa de atualização dos parâmetros. Usaremos o gradiente descendente, embora existam várias outras abordagens.

Para um parâmetro $\theta$, temos a regra para atualização:

$\theta = \theta - \alpha \dfrac{\partial J}{\partial \theta}$

In [7]:
def update_parameters(parameters, grads, learning_rate = 0.0001):
    """
    Atualiza os parâmetros utilizando o gradiente descendente
    
    Argumentos:
    parameters -- dicionário python contendo seus parâmetros 
    grads -- dicionário python contendo seus gradientes 
    
    Returns:
    parameters -- dicionário python contendo os parâmetros atualizados 
    """
    
    # Recupera cada parâmetro do dicionário "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Recupera cada gradiente do dicionário "grads"
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    # Regra de atualização para cada parâmetro
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

## Predição

Aqui iremos transformar as probabilidades da camada de saída em classes binárias. Vale ressaltar que o limite (_threshold_) utilizado é de 0,5, mas pode ser assumido diferentes valores, a depender da criticidade do problema, e o quão confiante você quer estar para rotular como 1 ou 0 seus exemplos.

In [8]:
def predict(parameters, X):
    """
    Usando os parâmetros aprendidos, prediz a classe de cada exemplo em X
    
    Argumentos:
    parameters -- dicionário python, contendo seus parâmetros 
    X -- dados de entrada de dimensões (n_x, m)
    
    Retorna
    predictions -- vetor de predições do modelo (0 ou 1)
    """
    # *********** AVISO SOBRE DIMENSÕES DAS MATRIZES *****************
    # As funções desenvolvidas consideram os arrays de dados X e y 
    # de dimensões (número atributos x número exemplos), ou seja, 
    # os atributos como as linhas e cada vetor de exemplo nas colunas.
    # Dessa forma, é necessária fazer a seguinte transformação:
    
    X = X.reshape(X.shape[0], -1).T
        
    # Calcula a probabilidade usando propagação direta e classifica em 0 ou 1 usando 0.5 como limiar (threshold).
    A2, cache = forward_propagation(X, parameters)
    predictions = A2 > 0.5
    
    return predictions

## Modelo de classificação

Depois de elencar cada etapa do processo, será mostrada adiante uma função que reúne esses passos e constrói o modelo de classificação.

In [9]:
def nn_model(X, Y, n_h, num_iterations = 10000, learning_rate = 0.0001, print_cost=False):
    """
    Argumentos:
    X -- dados de entrada de dimensões (2, número de exemplos)
    Y -- vetor de saída com rótulos, dimensões (1, número de exemplos)
    n_h -- tamanho da camada intermediária
    num_iterations -- número de iterações do loop do gradiente descendente
    print_cost -- se True, printa o custo a cada 1000 iterações
    
    Retorna:
    parameters -- parâmetros aprendidos pelo algoritmo. Podem ser usados para fazer previsões de dados não vistos.
    """

    # Considerando o aviso dado na função predict sobre as 
    # dimensões dos arrays X e Y, também será aplicado aqui
    # as transformações de linhas e colunas.
    X = X.reshape(X.shape[0], -1).T
    Y = Y.reshape(Y.shape[0], -1).T    
      
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Inicializa parâmetros
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # Loop (gradiente descendente)
    for i in range(0, num_iterations):
         
        # Propagação direta
        A2, cache = forward_propagation(X, parameters)
        
        # Função de custo
        cost = compute_cost(A2, Y)
 
        # Propagação reversa
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Atualização dos parâmetros pelo gradiente descendente
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Printa o custo a cada 1000 iterações
        if print_cost and i % 1000 == 0:
            print ("Custo depois da iteração %i: %f" %(i, cost))

    return parameters

## Exemplo de uso do modelo classificador

Agora, depois de construídas as funções do modelo, vamos usá-lo para um [dataset](https://archive.ics.uci.edu/ml/datasets/Haberman%27s%2BSurvival) simples (os valores das classes foram convertidos de 2->1 e 1->0).

In [10]:
df = pd.read_csv('haberman.data')

X = df.values[:, 0:-1]
y = df.values[:, -1]

seed = 2019

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, 
                                                    random_state = seed)

parameters = nn_model(X_train, y_train, 7, learning_rate = 0.001,
                      num_iterations = 5000, print_cost = True)

y_pred = predict(parameters, X_test)

print("Acurácia: %.4f"%(100 - np.mean(np.abs(y_pred - y_test)) * 100) + "%")

Custo depois da iteração 0: 1.174277
Custo depois da iteração 1000: 0.576007
Custo depois da iteração 2000: 0.562930
Custo depois da iteração 3000: 0.547394
Custo depois da iteração 4000: 0.538858
Acurácia: 76.6234%


## Referências:
- Redes Neurais Artificiais: Teoria e aplicações
- Neural Networks and Deep Learning (Coursera - deeplearning.ai)